# Packages Installs, imports, and presets

In [256]:
#from scapy.all import *
import pandas as pd
#import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
from collections import defaultdict
import os
from transformers import MobileViTV2Model, MobileViTV2Config
import torch
import torchmetrics
from torch import optim
import lightning as L
import torch
import torch.nn as nn

In [257]:
from lightning.pytorch.loggers import WandbLogger
import wandb
wandb.init() 

lr-AdamW,▁
trainer/global_step,▁
lr-AdamW,0.0001
trainer/global_step,0


In [258]:
INPUT_SIZE = (256,256,3)

# Feature engineering and Loading

In [259]:
TIMEOUT = 30

In [260]:
df_train= pd.read_parquet(f'data/iec104_train_{TIMEOUT}.parquet')
df_validation = pd.read_parquet(f'data/iec104_validation_{TIMEOUT}.parquet')
df_test=pd.read_parquet(f'data/iec104_test_{TIMEOUT}.parquet')

In [261]:

print('train, validation,test (shape):',df_train.shape,df_validation.shape,df_test.shape)
print('train, validation,test (number of flows):',df_train.index.nunique(),df_validation.index.nunique(),df_test.index.nunique())

train, validation,test (shape): (383703, 30) (32635, 30) (108734, 30)
train, validation,test (number of flows): 9757 1085 2711


In [262]:
def encode_time(df):
    df['time'] = pd.to_datetime(df['time'], unit='s')
    df['hour'] = df['time'].dt.hour
    df['minute'] = df['time'].dt.minute
    df['second'] = df['time'].dt.second

    df['hour_sin'] = np.sin(2 * np.pi * df['hour'] / 24)
    df['hour_cos'] = np.cos(2 * np.pi * df['hour'] / 24)

    # Encode minute cyclically
    df['minute_sin'] = np.sin(2 * np.pi * df['minute'] / 60)
    df['minute_cos'] = np.cos(2 * np.pi * df['minute'] / 60)

    # Encode second cyclically
    df['second_sin'] = np.sin(2 * np.pi * df['second'] / 60)
    df['second_cos'] = np.cos(2 * np.pi * df['second'] / 60)
    df.drop('time',axis=1,inplace=True)
encode_time(df_train)
encode_time(df_validation)
encode_time(df_test)

In [263]:
from feature_engine.encoding import OrdinalEncoder
od = OrdinalEncoder(encoding_method='arbitrary')

feats=['ip_flags','ip_src','ip_dst','tcp_flags','label','tcp_sport', 'tcp_dport']

od.fit(df_train[feats])
df_train[feats] = od.transform(df_train[feats])
df_validation[feats] = od.transform(df_validation[feats])
df_test[feats] = od.transform(df_test[feats])

In [264]:
cols_to_drop = df_train.columns[df_train.nunique() == 1]
print(cols_to_drop)
# Drop these columns from the DataFrame
df_train = df_train.drop(cols_to_drop, axis=1)
df_validation = df_validation.drop(cols_to_drop, axis=1)
df_test = df_test.drop(cols_to_drop, axis=1)

Index(['ip_version', 'ip_ihl', 'ip_tos', 'ip_flags', 'ip_frag', 'ip_proto',
       'ip_options', 'tcp_reserved', 'tcp_flags', 'tcp_urgptr'],
      dtype='object')


In [265]:
df_train.columns

Index(['packet_number', 'ip_len', 'ip_id', 'ip_ttl', 'ip_chksum', 'ip_src',
       'ip_dst', 'tcp_sport', 'tcp_dport', 'tcp_seq', 'tcp_ack', 'tcp_dataofs',
       'tcp_window', 'tcp_chksum', 'tcp_options', 'size', 'payload',
       'payload_raw', 'label', 'hour', 'minute', 'second', 'hour_sin',
       'hour_cos', 'minute_sin', 'minute_cos', 'second_sin', 'second_cos'],
      dtype='object')

In [266]:
from feature_engine.wrappers import SklearnTransformerWrapper
from sklearn.preprocessing import MinMaxScaler,StandardScaler
numeric_features = [
    'ip_len', 'ip_id', 'ip_ttl', 'ip_chksum',  'tcp_seq', 'tcp_ack',
    'tcp_dataofs', 'tcp_window', 'tcp_chksum', 'size'
]

# Initialize MinMaxScaler
# scaler = SklearnTransformerWrapper(StandardScaler())
# scaler.fit(df_train[numeric_features])
# # Fit and transform the DataFrame
# df_train[numeric_features] = scaler.transform(df_train[numeric_features])
# df_test[numeric_features] = scaler.transform(df_test[numeric_features])
# df_validation[numeric_features] = scaler.transform(df_validation[numeric_features])

In [267]:
# def reset_indexes(df):
#     return df.reset_index('packet_number',inplace=True)
# reset_indexes(df_train)
# reset_indexes(df_test)
# reset_indexes(df_validation)

In [268]:
df_train.shape

(383703, 28)

In [269]:
# df_train[df_train.dtypes=='object']

In [270]:
display(df_train['label'].value_counts())
display(df_validation['label'].value_counts())
display(df_test['label'].value_counts())

label
1     160434
0     138093
9      14507
8      14489
11     11878
6       8424
10      7732
5       7585
7       6685
3       5113
4       4776
2       3987
Name: count, dtype: int64

label
1     13933
0      8811
8      1611
9      1450
11     1211
10     1165
6       967
5       847
4       739
7       695
2       629
3       577
Name: count, dtype: int64

label
1     58283
0     26311
8      4162
9      3939
11     3405
6      2276
5      2056
10     1950
7      1926
4      1679
2      1410
3      1337
Name: count, dtype: int64

In [271]:
df_train

,packet_number,ip_len,ip_id,ip_ttl,ip_chksum,ip_src,ip_dst,tcp_sport,tcp_dport,tcp_seq,...,label,hour,minute,second,hour_sin,hour_cos,minute_sin,minute_cos,second_sin,second_cos
flow,,,,,,,,,,,,,,,,,,,,,
"[1, '192.168.1.19', '192.168.1.29', 2404, 44581, 'TCP', 'c_se_na_1_DoS']",0,58,22237,64,24672,0,0,44581,2404,2021378529,...,10,9,51,28,0.707107,-0.707107,-0.809017,0.587785,0.207912,-0.978148
"[1, '192.168.1.19', '192.168.1.29', 2404, 44581, 'TCP', 'c_se_na_1_DoS']",1,58,5610,128,24915,1,1,2404,44581,3064358417,...,10,9,51,28,0.707107,-0.707107,-0.809017,0.587785,0.207912,-0.978148
"[1, '192.168.1.19', '192.168.1.29', 2404, 44581, 'TCP', 'c_se_na_1_DoS']",2,68,5613,128,24902,1,1,2404,44581,3064358423,...,10,9,51,31,0.707107,-0.707107,-0.809017,0.587785,-0.104528,-0.994522
"[1, '192.168.1.19', '192.168.1.29', 2404, 44581, 'TCP', 'c_se_na_1_DoS']",3,68,22240,64,24659,0,0,44581,2404,2021378535,...,10,9,51,41,0.707107,-0.707107,-0.809017,0.587785,-0.913545,-0.406737
"[1, '192.168.1.19', '192.168.1.29', 2404, 44581, 'TCP', 'c_se_na_1_DoS']",4,70,5620,128,24893,1,1,2404,44581,3064358439,...,10,9,51,41,0.707107,-0.707107,-0.809017,0.587785,-0.913545,-0.406737
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"[1, '192.168.1.21', '192.168.1.28', 2404, 34187, 'TCP', 'c_rp_na_1_DoS']",5,68,11813,64,35085,5,8,34187,2404,3523926085,...,9,17,42,10,-0.965926,-0.258819,-0.951057,-0.309017,0.866025,0.500000
"[1, '192.168.1.21', '192.168.1.28', 2404, 34187, 'TCP', 'c_rp_na_1_DoS']",6,67,29917,128,598,8,6,2404,34187,1743562132,...,9,17,42,10,-0.965926,-0.258819,-0.951057,-0.309017,0.866025,0.500000
"[1, '192.168.1.21', '192.168.1.28', 2404, 34187, 'TCP', 'c_rp_na_1_DoS']",7,58,11815,64,35093,5,8,34187,2404,3523926101,...,9,17,42,20,-0.965926,-0.258819,-0.951057,-0.309017,0.866025,-0.500000


In [272]:
tmp=df_train.groupby('flow')['ip_id'].count().value_counts()
tmp=tmp.sort_index().cumsum()
tmp = tmp/tmp.max()
display(tmp)

ip_id
2        0.000615
3        0.000717
4        0.000820
5        0.000922
6        0.004612
           ...   
8808     0.999590
9579     0.999693
25497    0.999795
25537    0.999898
27265    1.000000
Name: count, Length: 111, dtype: float64

In [273]:
# df_train.drop('payload_raw')

In [274]:
MAX_PAYLOAD_LENGTH = df_train['payload_raw'].apply(len).max()
def payload_to_binary_columns(df, column_name):
    """
    Converts a column with binary data into separate bit columns.

    Args:
        df (pd.DataFrame): The input DataFrame.
        column_name (str): The name of the column to convert.

    Returns:
        pd.DataFrame: The transformed DataFrame with bit columns.
    """
    # Find the maximum length of the payload
    # max_payload_length = df[column_name].apply(len).max()
    bit_columns = ['bit_' + str(i) for i in range(MAX_PAYLOAD_LENGTH * 8)]
    
    def bytes_to_bits(byte_data):
        byte_array = np.frombuffer(byte_data, dtype=np.uint8)
        return np.unpackbits(byte_array)
    
    # Convert and pad the binary data
    binary_data = df[column_name].apply(lambda x: np.pad(bytes_to_bits(x), (0, MAX_PAYLOAD_LENGTH * 8 - len(x) * 8), 'constant'))

    # Stack the binary data into a matrix
    binary_matrix = np.vstack(binary_data.values)
    
    # Create a DataFrame from the binary matrix
    binary_df = pd.DataFrame(binary_matrix, columns=bit_columns)
    df=df.reset_index()
    # Concatenate the binary DataFrame with the original DataFrame
    return pd.concat([df.drop(columns=[column_name]), binary_df], axis=1).set_index('flow')
# Transform the DataFrame


df_train.drop('payload_raw',axis=1,inplace=True)
df_test.drop('payload_raw',axis=1,inplace=True)

df_validation.drop('payload_raw',axis=1,inplace=True)

# df_train=payload_to_binary_columns(df_train, 'payload_raw')
# df_validation=payload_to_binary_columns(df_validation, 'payload_raw')
# df_test=payload_to_binary_columns(df_test, 'payload_raw')

# Training Dataset

In [275]:
import torch.utils.data as data_utils
import torch.nn.functional as F
from torch.utils.data import Dataset


class IEC104Dataset(Dataset):
    def __init__(self, df, transform=None, target_transform=None):
        self.df = df
        self.flow_int_id = list(set(self.df.index))
        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):
        return len(self.flow_int_id)

    def __getitem__(self, idx):
        df_idx = self.flow_int_id[idx]
        flow = self.df.loc[df_idx, self.df.columns != 'label']
        label = self.df.loc[df_idx, 'label']

        if isinstance(flow, pd.Series):
            flow = flow.to_frame().T
        
        # if isinstance(label, np.float64):
        #     print('label:',label)
        if isinstance(label,np.float64):
            label = label.astype(int)
        elif not isinstance(label, np.int64):
            label = label.iloc[0]
        label = torch.tensor(label)

        if self.transform:
            flow = self.transform(flow)
        if self.target_transform:
            label = self.target_transform(label)
        # print(flow)
        # print(label)
        return flow, label

def feature_transform(flow):
    # Convert flow to a DataFrame if it's not already
    flow = pd.DataFrame(flow)

    # Convert to tensors
    dynamic_tensor = torch.tensor(flow.values, dtype=torch.float32)

    if dynamic_tensor.shape[0] > SEQ_LEN:
        # Randomly select a starting index for the sequence
        start_idx = np.random.randint(0, dynamic_tensor.shape[0] - SEQ_LEN + 1)
        dynamic_tensor = dynamic_tensor[start_idx:start_idx + SEQ_LEN]
    else:
        # If the tensor is shorter than SEQ_LEN, pad with -1 as before
        X = torch.zeros(SEQ_LEN, dynamic_tensor.shape[1], dtype=torch.float32)
        min_shape_0 = min(dynamic_tensor.shape[0], SEQ_LEN)
        X[:min_shape_0] = dynamic_tensor[-min_shape_0:]
        dynamic_tensor = X
    # dynamic_tensor = (dynamic_tensor+1)/2
    return dynamic_tensor

    # return dynamic_tensor


### Dataset Loader

# Model Design

In [276]:
# model_vitv2_output_shape = [ 512, 8, 8]
NUM_CLASSES = df_train.label.nunique()

In [277]:
def get_device():
    device = "cpu"
    if torch.cuda.is_available():
      device = "cuda:1"
    return device
DEVICE = get_device()

In [278]:
import seaborn as sns

In [279]:
import wandb

### Lightning

In [280]:
class LFVIT(L.LightningModule):
    def __init__(self, model,name):
        super().__init__()
        self.model = model
        self.name = name
        self.step_outputs = defaultdict(list)
        self.step_target = defaultdict(list)
    def forward(self, inputs):
        return self.model(inputs)
    def training_step(self, batch, batch_idx):
        x, y = batch
        # print(x.shape)
        target_prediction = self.model(x)
        loss = nn.functional.cross_entropy(target_prediction, y)
        self.step_outputs['train'].append(target_prediction)
        self.step_target['train'].append(y)
        self.log('train_loss',loss,on_epoch=True,on_step=False)
        return loss
    def validation_step(self, batch, batch_idx):
        x, y = batch
        # print(x.shape)

        target_prediction = self.model(x)
        self.step_outputs['validation'].append(target_prediction)
        self.step_target['validation'].append(y)
        loss = nn.functional.cross_entropy(target_prediction, y)

        self.log('val_loss',loss,on_epoch=True,on_step=False)
        # loss = nn.functional.cross_entropy(target_prediction, y)
        # self.log('val_loss',loss)
    def test_step(self, batch, batch_idx):
        x, y = batch
        target_prediction = self.model(x)
        self.step_outputs['test'].append(target_prediction)
        self.step_target['test'].append(y)       
    # def on_test_epoch_end(self):
    # def on_train_epoch_end(self):

    def on_test_epoch_end(self):
        self._shared_on_epoch_end(end_type='test')
    def on_validation_epoch_end(self):
        self._shared_on_epoch_end('validation')
    def _shared_on_epoch_end(self,end_type):
        all_preds = torch.cat(self.step_outputs[end_type])
        all_targets = torch.cat(self.step_target[end_type])
        metrics = dict()
        suffix= end_type+'_'    
        accuracy = torchmetrics.Accuracy(task="multiclass", num_classes=NUM_CLASSES,average='macro').to(DEVICE)
        metrics[suffix+'accuracy'] = accuracy(all_preds, all_targets)
         
        metric = torchmetrics.Precision(task="multiclass", num_classes=NUM_CLASSES,average='macro').to(DEVICE)
        metrics[suffix+'precision'] = metric(all_preds, all_targets)
        
        metric = torchmetrics.Recall(task="multiclass", num_classes=NUM_CLASSES,average='macro').to(DEVICE)
        metrics[suffix+'recall'] = metric(all_preds, all_targets) 
        
        stat_scores = torchmetrics.classification.MulticlassStatScores(num_classes=NUM_CLASSES,average='micro').to(DEVICE)

        # Update the state with the true and predicted labels
        stat_scores.update(all_preds, all_targets)

        # Compute the scores
        scores = stat_scores.compute()

        # Extract TP, FP, TN, FN for each class
        tp = scores[ 0]
        fp = scores[ 1]
        tn = scores[ 2]
        fn = scores[ 3]

        # Compute TPR and FPR for each class
        tpr = tp / (tp + fn)
        fpr = fp / (fp + tn)
        metrics[suffix+'tpr'] = tpr
           
        metrics[suffix+'fpr'] = fpr
        confusionmatrix = torchmetrics.ConfusionMatrix(task="multiclass", num_classes=NUM_CLASSES).to(DEVICE)
        confusionmatrix_result = confusionmatrix(all_preds, all_targets).cpu().numpy()
        df_cm = pd.DataFrame(confusionmatrix_result, index = range(NUM_CLASSES), columns=range(NUM_CLASSES))
        # metrics[suffix+'cm'] = str(confusionmatrix_result)
        plt.figure(figsize = (10,7))
        fig_ = sns.heatmap(df_cm, annot=True, cmap='Spectral').get_figure()
        plt.close(fig_)
        fig_.savefig('tmp/confusionmatrix.png')
        wandb_logger.log_image(key=suffix+"confusionmatrix", images=['tmp/confusionmatrix.png'])
        
        confusionmatrix = torchmetrics.ConfusionMatrix(normalize='true',task="multiclass", num_classes=NUM_CLASSES).to(DEVICE)
        confusionmatrix_result = confusionmatrix(all_preds, all_targets).cpu().numpy()
        df_cm = pd.DataFrame(confusionmatrix_result, index = range(NUM_CLASSES), columns=range(NUM_CLASSES))
        plt.figure(figsize = (10,7))
        fig_ = sns.heatmap(df_cm, annot=True, cmap='Spectral').get_figure()
        plt.close(fig_)
        fig_.savefig('tmp/confusionmatrix_normalized.png')
        wandb_logger.log_image(key=suffix+"confusionmatrix_normalized", images=['tmp/confusionmatrix_normalized.png'])
        
        self.log_dict(metrics)
        self.step_outputs[end_type].clear()
        self.step_target[end_type].clear()
    def configure_optimizers(self):
        optimizer = optim.AdamW(self.parameters(), lr=1e-4)
        scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer,patience=3,mode='min')
        return {'optimizer':optimizer,"lr_scheduler":{"scheduler": scheduler, "interval": "epoch","frequency": 1,"monitor": "val_loss"}}


In [281]:

# accuracy = torchmetrics.Accuracy(task="multiclass", num_classes=NUM_CLASSES).to(DEVICE)

# accuracy(torch.tensor(4, device='cuda:0'),
# torch.tensor([0.0824, 0.0977, 0.1032, 0.0896, 0.0766, 0.1045, 0.0956, 0.0752, 0.0876,
#         0.0868, 0.1010], device='cuda:0'))

###  FVIT


In [282]:
import math

class FVIT(nn.Module):
    def __init__(self,encoder,decoder):
        super(FVIT, self).__init__()
        self.encoder = encoder
        self.decoder = decoder

    def forward(self, x):
        hidden_states=self.encoder(x[0])
        concatenated_input = torch.cat((torch.flatten(hidden_states, start_dim=1), torch.flatten(x[1], start_dim=1)),dim=1)
        return self.decoder(concatenated_input)
    
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len=5000):
        super(PositionalEncoding, self).__init__()
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0)  # Shape: (1, max_len, d_model)
        # self.pe = pe.to(DEVICE)
        
        self.register_buffer('pe', pe)

    def forward(self, x):
        # x shape: (batch_size, seq_len, d_model)
        # print(x.shape)
        # print('pedevice',self.pe.device)
        x = x + self.pe[:, :x.size(1), :]  # Shape: (batch_size, seq_len, d_model)
        return x

class TransformerModel(nn.Module):
    def __init__(self,d_model, nhead, nhid, nlayers, dropout):
        super(TransformerModel, self).__init__()
        self.model_type = 'Transformer'
        self.pos_encoder = PositionalEncoding(d_model,SEQ_LEN)
        # print(d_model,nhead,dropout,nhid)
        self.transformer_encoder = nn.TransformerEncoder(
                        nn.TransformerEncoderLayer(d_model=d_model, nhead=nhead, dropout=dropout,dim_feedforward=nhid,activation=nn.ReLU(),batch_first=True),
                num_layers=nlayers,norm=nn.LayerNorm(normalized_shape=NUM_FEATURES)
                )
        self.d_model = d_model
        self.decoder = nn.Sequential(
            nn.Flatten(),
            nn.Linear(NUM_FEATURES*SEQ_LEN, NUM_CLASSES),
            # nn.Dropout(0.5),
            # nn.ReLU(),
            # nn.Linear(100, NUM_CLASSES),
            nn.Softmax(dim=1)
)   
    # def init_weights(self):
    #     initrange = 0.1
    #     self.transformer_encoder.bias.data.zero_()
    #     self.transformer_encoder.weight.data.uniform_(-initrange, initrange)
        
    def forward(self, x):
        # print('fow',x.shape)
        x = self.pos_encoder(x)
        x = self.transformer_encoder(x)
        # print(x.shape)
        # print("DD:",x)
        x= self.decoder(x)
        # return output
        return x

In [283]:
# df_train.shape[1]-1,df_validation.shape[1]-1,df_test.shape[1]-1

In [284]:
NUM_FEATURES = df_train.shape[1]-1
NUM_FEATURES

26

In [285]:
output_dim_transformer_encoder = [SEQ_LEN,SEQ_LEN,NUM_FEATURES]

In [286]:
NUM_FEATURES

26

In [287]:
from x_transformers import Encoder

In [288]:
SEQ_LEN = 20
BATCH_SIZE=128
NUM_WORKERS =6
PREFETCH_FACTOR=2
train_dataset = IEC104Dataset(df_train,transform = feature_transform)
validation_dataset = IEC104Dataset(df_validation,transform = feature_transform)
test_dataset = IEC104Dataset(df_test,transform = feature_transform)


class_sample_output=df_train.reset_index().groupby('label')['flow'].nunique().tolist()
# print(class_sample_output)
sampling_weights = 1 / torch.Tensor(class_sample_output)
# print(sampling_weights)

labels = train_dataset.df.groupby('flow')['label'].unique()
# print(train_dataset.flow_int_id[0])
labels=[labels.loc[i][0] for i in train_dataset.flow_int_id]
samples_weight = np.array([sampling_weights[t] for t in labels]).flatten()

samples_weight = torch.from_numpy(samples_weight)
samples_weigth = samples_weight.double()
# print(samples_weigth)
sampler = torch.utils.data.sampler.WeightedRandomSampler(samples_weight, len(samples_weight))


# train_loader = data_utils.DataLoader(dataset = train_dataset, batch_size = BATCH_SIZE,num_workers=NUM_WORKERS,prefetch_factor=PREFETCH_FACTOR)
train_loader = data_utils.DataLoader(dataset = train_dataset, batch_size = BATCH_SIZE,num_workers=NUM_WORKERS,prefetch_factor=PREFETCH_FACTOR,sampler = sampler)#,collate_fn = collate_fn)
# train_loader = data_utils.DataLoader(dataset = train_dataset, batch_size = BATCH_SIZE,num_workers=NUM_WORKERS,prefetch_factor=PREFETCH_FACTOR)
# next(iter(train_loader))

validation_loader = data_utils.DataLoader(dataset = validation_dataset, batch_size = BATCH_SIZE,num_workers=NUM_WORKERS,prefetch_factor=PREFETCH_FACTOR)#,collate_fn = collate_fn)
test_loader = data_utils.DataLoader(dataset = test_dataset, batch_size = BATCH_SIZE,num_workers=NUM_WORKERS,prefetch_factor=PREFETCH_FACTOR)#,collate_fn = collate_fn)


In [289]:
class BinaryEncodingTransformerModel(nn.Module):
    def __init__(self):
        super(BinaryEncodingTransformerModel, self).__init__()
        self.model_type = 'Transformer'
 
        
        self.decoder = nn.Sequential(
    PositionalEncoding(NUM_FEATURES,SEQ_LEN),
    Encoder(dim = NUM_FEATURES,
        depth = 6,
        heads = 4,
           ff_dropout = 0.2,          # feed-forward dropout rate
    attn_dropout = 0.2,         # attention dropout rate
    # attn_sparse_topk = 8
        ),
            nn.Flatten(),
            nn.Linear(NUM_FEATURES*SEQ_LEN, NUM_CLASSES),
     
            
            nn.Softmax(dim=1))

    def forward(self, x):

        x = self.decoder(x)
        return x



class GRUModel(nn.Module):
    def __init__(self, num_features, num_classes, seq_len):
        super(GRUModel, self).__init__()
        self.gru = nn.GRU(num_features, 10*num_features, 3, dropout = 0.2,batch_first=True)
        self.linear = nn.Linear(10*num_features * seq_len, num_classes)
        self.softmax = nn.Softmax(dim=1)
    
    def forward(self, x):
        gru_out, _ = self.gru(x)  # Get the output and hidden state from GRU
        gru_out = gru_out.contiguous().view(gru_out.size(0), -1)  # Flatten the output
        linear_out = self.linear(gru_out)
        output = self.softmax(linear_out)
        return output


class RNNModel(nn.Module):
    def __init__(self, num_features, num_classes, seq_len):
        super(RNNModel, self).__init__()
        self.rnn = nn.RNN(num_features, 1000, 3, dropout=0.2, batch_first=True,bidirectional=False,nonlinearity='tanh')
        self.linear = nn.Linear(1000 * seq_len, num_classes)
        self.softmax = nn.Softmax(dim=1)
    
    def forward(self, x):
        rnn_out, _ = self.rnn(x)  # Get the output and hidden state from RNN
        rnn_out = rnn_out.contiguous().view(rnn_out.size(0), -1)  # Flatten the output
        linear_out = self.linear(rnn_out)
        output = self.softmax(linear_out)
        return output



import torch.nn as nn

# Assuming these classes and modules are already defined:
# PositionalEncoding, Encoder, LFVIT, GRUModel, RNNModel

def create_model(model_name):
    if model_name == "Transformer":
        model = LFVIT(nn.Sequential(
            # nn.Linear(NUM_FEATURES*SEQ_LEN, 50*SEQ_LEN),
            PositionalEncoding(NUM_FEATURES, SEQ_LEN),
            Encoder(
                dim=NUM_FEATURES,
                ff_mult=16,
                depth=6,
                heads=6,
                ff_dropout=0.2,          # feed-forward dropout rate
                attn_dropout=0.2,        # attention dropout rate
                # ff_hidden_act=nn.GELU(),
                ff_glu=True,
                # attn_sparse_topk=8
            ),
            nn.Flatten(),
            nn.Linear(NUM_FEATURES * SEQ_LEN, NUM_CLASSES),
            # nn.Linear(NUM_FEATURES*SEQ_LEN, 1000),
            # nn.ReLU(),
            # nn.Linear(1000, 200),
            # nn.ReLU(),
            # nn.Dropout(0.1),
            # nn.Linear(200, NUM_CLASSES),
            nn.Softmax(dim=1)
        ), "Transformer-HEADER")
    
    elif model_name == "GRU":
        model = LFVIT(GRUModel(NUM_FEATURES, NUM_CLASSES, SEQ_LEN), "GRU-HEADER")
    
    elif model_name == "RNN":
        model = LFVIT(RNNModel(NUM_FEATURES, NUM_CLASSES, SEQ_LEN), "RNN-HEADER")
    
    elif model_name == "SEQ-MLP":
        layer_sizes = [NUM_FEATURES * SEQ_LEN, NUM_FEATURES * SEQ_LEN, 512, NUM_CLASSES]
        layers = [nn.Flatten()]  # Add flatten layer at the beginning
        for i in range(len(layer_sizes) - 1):
            layers.append(nn.Linear(layer_sizes[i], layer_sizes[i + 1]))
            layers.append(nn.GELU())  # Activation function
            layers.append(nn.Dropout(0.1))  # Dropout with a rate of 0.1
        layers.append(nn.Softmax(dim=1))
        fcn = nn.Sequential(*layers)
        model = LFVIT(fcn, "SEQ-MLP-HEADER")
    
    else:
        raise ValueError(f"Unknown model name: {model_name}")

    return model

# Example usage
model_fvit = create_model("GRU")
# or
# model_fvit = create_model("GRU")
# or
# model_fvit = create_model("RNN")
# or
# model_fvit = create_model("SEQ-MLP")


In [290]:
model_fvit.model

GRUModel(
  (gru): GRU(26, 260, num_layers=3, batch_first=True, dropout=0.2)
  (linear): Linear(in_features=5200, out_features=12, bias=True)
  (softmax): Softmax(dim=1)
)

## Running


In [291]:
from lightning.pytorch.callbacks import LearningRateMonitor

In [292]:
wandb.finish()
wandb_logger = WandbLogger(log_model="all")

MAX_EPOCHS=20
# timer =  L.pytorch.calalbacks.Timer()
def run_fvit_trainer_fit(train_loader,validation_loader,model):
    checkpoint_callback = L.pytorch.callbacks.ModelCheckpoint(
        monitor='train_loss',
        filename='iec104-{epoch:02d}-{loss}',
        save_top_k=1,
        mode='min', 
    )
    lr_monitor = LearningRateMonitor(logging_interval='epoch')
    # early_stop_callback = L.pytorch.callbacks.early_stopping.EarlyStopping(monitor="val_loss", min_delta=0.001, patience=3, verbose=False, mode="min")
    # cpb=CustomProgressBar()
    trainer = L.Trainer(max_epochs=MAX_EPOCHS,callbacks=[checkpoint_callback,lr_monitor],accelerator='cuda',
                        devices=[1],log_every_n_steps=20
                        #,check_val_every_n_epoch=1
                        # ,limit_train_batches=100,limit_val_batches=100#,max_time={"minutes": 60}
                        ,logger=wandb_logger)
    trainer.fit(model=model, train_dataloaders=train_loader,val_dataloaders=validation_loader)
    return checkpoint_callback,trainer
checkpoint_callback,trainer=run_fvit_trainer_fit(
    train_loader,
    validation_loader,
     model=model_fvit)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]

  | Name  | Type     | Params
-----------------------------------
0 | model | GRUModel | 1.1 M 
-----------------------------------
1.1 M     Trainable params
0         Non-trainable params
1.1 M     Total params
4.405     Total estimated model params size (MB)


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

In [ ]:
trainer.test(ckpt_path="best",dataloaders=test_loader)

Restoring states from the checkpoint path at ./lightning_logs/jru54kaz/checkpoints/iec104-epoch=09-loss=0.ckpt
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]
Loaded model weights from the checkpoint at ./lightning_logs/jru54kaz/checkpoints/iec104-epoch=09-loss=0.ckpt


Testing: |          | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│       test_accuracy       │    0.2369232475757599     │
│         test_fpr          │    0.07236511260271072    │
│      test_precision       │    0.1299554407596588     │
│        test_recall        │    0.2369232475757599     │
│         test_tpr          │    0.20398376882076263    │
└───────────────────────────┴───────────────────────────┘

[{'test_accuracy': 0.2369232475757599,
  'test_precision': 0.1299554407596588,
  'test_recall': 0.2369232475757599,
  'test_tpr': 0.20398376882076263,
  'test_fpr': 0.07236511260271072}]

In [ ]:
best_checkpoint_path = checkpoint_callback.best_model_path
print(f"Best checkpoint path: {best_checkpoint_path}")
best_model= LFVIT.load_from_checkpoint(best_checkpoint_path,model=model_fvit.model,name=model_fvit.name)
best_model.to(DEVICE)

Best checkpoint path: ./lightning_logs/jru54kaz/checkpoints/iec104-epoch=09-loss=0.ckpt


LFVIT(
  (model): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Linear(in_features=520, out_features=256, bias=True)
    (2): GELU(approximate='none')
    (3): Dropout(p=0.1, inplace=False)
    (4): Linear(in_features=256, out_features=128, bias=True)
    (5): GELU(approximate='none')
    (6): Dropout(p=0.1, inplace=False)
    (7): Linear(in_features=128, out_features=64, bias=True)
    (8): GELU(approximate='none')
    (9): Dropout(p=0.1, inplace=False)
    (10): Linear(in_features=64, out_features=32, bias=True)
    (11): GELU(approximate='none')
    (12): Dropout(p=0.1, inplace=False)
    (13): Linear(in_features=32, out_features=12, bias=True)
    (14): GELU(approximate='none')
    (15): Dropout(p=0.1, inplace=False)
    (16): Softmax(dim=1)
  )
)

In [ ]:
import time
from constants import NUMBER_PACKETS_OPTIONS
import constants

In [ ]:
DEVICE

'cuda:1'

In [ ]:
# best_model = model_fvit

# best_checkpoint_path = checkpoint_callback.best_model_path
# print(f"Best checkpoint path: {best_checkpoint_path}")
# best_model = LFVIT.load_from_checkpoint(best_checkpoint_path,model=GRUModel(NUM_FEATURES, NUM_CLASSES, SEQ_LEN),name='GRU')

In [ ]:
def run_experiment(df,num_packets):
    if num_packets != 'full':
        df = df[df['packet_number'].isin(list(range(num_packets)))]
        df = df.groupby(level='flow').filter(lambda x: len(x) <= num_packets)
    test_dataset = IEC104Dataset(df,transform = feature_transform)
    test_loader = data_utils.DataLoader(dataset = test_dataset, batch_size = 1,num_workers=NUM_WORKERS,prefetch_factor=PREFETCH_FACTOR)#,collate_fn = collate_fn)

    # test_dataset = IEC104Dataset(df_test,transform = feature_transform)
    # test_loader = data_utils.DataLoader(dataset = test_dataset, batch_size = 1,num_workers=NUM_WORKERS,prefetch_factor=PREFETCH_FACTOR)


    

    # Run predictions and measure time
    # all_preds = []
    prediction_times=[]
    start_time = time.time()
    best_model.to(DEVICE)
    best_model.eval()
    with torch.no_grad():
        for batch in test_loader:
            inputs, _ = batch
            inputs= inputs.to(DEVICE)
            batch_start_time = time.time()
            # print(inputs.shape)
            preds = best_model(inputs)
            batch_end_time = time.time()
            batch_time = batch_end_time - batch_start_time
            prediction_times.append(batch_time)

            # print(f"Batch prediction time: {batch_time:.4f} seconds")
            # all_preds.append(preds)
    prediction_times= pd.DataFrame(prediction_times)
    metrics_results = trainer.test(best_model,dataloaders=test_loader)
    metrics_results= metrics_results[0]
    formatted_metrics = constants.metrics_log_formatter(model_name, "", num_packets, TIMEOUT,
        accuracy=metrics_results['test_accuracy'],
                                          precision=metrics_results['test_precision'],
                                          recall=metrics_results['test_recall'],
                                          cm=None,
                                          tpr_micro=metrics_results['test_tpr'],
                                          fpr_micro=metrics_results['test_fpr'],
                                          avg_pred_time=prediction_times.mean().item(),
                                          std_pred_time=prediction_times.std().item(),
                                          avg_throughput=1/prediction_times.mean().item(),
                                          std_throughput=(1/prediction_times).std().item(),
                                          avg_total_pred_time=prediction_times.mean().item(),
                                          std_total_pred_time=prediction_times.std().item(),
                                          avg_total_throughput=1/prediction_times.mean().item(),
                                          std_total_throughput=(1/prediction_times).std().item())

    return formatted_metrics
    # total_time = time.time() - start_time
    # print(f"Total prediction time for the dataset: {total_time:.4f} seconds")

    # # Flatten all predictions into a single tensor
    # all_preds = torch.cat(all_preds, dim=0)

    # print(f"Predictions shape: {all_preds.shape}")


model_name = best_model.name

results = []
full_packets = 'full'
res= run_experiment(df_test,full_packets)
# performance_metrics =constants.metadata_log_formatter(model_name, "", full_packets, TIMEOUT)
# res = evaluate_model_performance(best_model, X_tests[number_packets], y_tests[number_packets], NUM_RUNS)
# performance_metrics.update(res)
results.append(res)

for number_packets in NUMBER_PACKETS_OPTIONS:
    # df_test=pd.read_parquet(f'data/iec104_test_{TIMEOUT}.parquet')
    res= run_experiment(df_test,number_packets)
    # performance_metrics = constants.metadata_log_formatter(model_name, "", number_packets, TIMEOUT)
    # res = evaluate_model_performance(best_model, X_tests[number_packets], y_tests[number_packets], NUM_RUNS)
    # performance_metrics.update(res)
    results.append(res)

df_results = pd.DataFrame(results)

output_path= 'data/results_fids.json'
df_results.to_json(output_path, mode='a',orient="records", lines=True)#, header=not os.path.exists(output_path))

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]


Testing: |          | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│       test_accuracy       │    0.2369232475757599     │
│         test_fpr          │    0.07236511260271072    │
│      test_precision       │    0.1299554407596588     │
│        test_recall        │    0.2369232475757599     │
│         test_tpr          │    0.20398376882076263    │
└───────────────────────────┴───────────────────────────┘

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]


Testing: |          | 0/? [00:00<?, ?it/s]

...

Testing: |          | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│       test_accuracy       │    0.2369232475757599     │
│         test_fpr          │    0.07236511260271072    │
│      test_precision       │    0.1299554407596588     │
│        test_recall        │    0.2369232475757599     │
│         test_tpr          │    0.20398376882076263    │
└───────────────────────────┴───────────────────────────┘

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]


Testing: |          | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│       test_accuracy       │    0.2369232475757599     │
│         test_fpr          │    0.07236511260271072    │
│      test_precision       │    0.1299554407596588     │
│        test_recall        │    0.2369232475757599     │
│         test_tpr          │    0.20398376882076263    │
└───────────────────────────┴───────────────────────────┘